In [1]:
import sys
import os

project_root = os.path.abspath("..")
src_path = os.path.join(project_root, "src")

if src_path not in sys.path:
    sys.path.insert(0, src_path)

In [2]:
import numpy as np

# === Shifts ===
def shift_right(mat, steps=1):
    return np.roll(mat, shift=steps, axis=1)

def shift_left(mat, steps=1):
    return np.roll(mat, shift=-steps, axis=1)

def shift_down(mat, steps=1):
    return np.roll(mat, shift=steps, axis=0)

def shift_up(mat, steps=1):
    return np.roll(mat, shift=-steps, axis=0)

def shift_diag_down_right(mat, steps=1):
    return np.roll(np.roll(mat, shift=steps, axis=0), shift=steps, axis=1)

def shift_diag_down_left(mat, steps=1):
    return np.roll(np.roll(mat, shift=steps, axis=0), shift=-steps, axis=1)

def shift_diag_up_right(mat, steps=1):
    return np.roll(np.roll(mat, shift=-steps, axis=0), shift=steps, axis=1)

def shift_diag_up_left(mat, steps=1):
    return np.roll(np.roll(mat, shift=-steps, axis=0), shift=-steps, axis=1)

# === New: Rotations (in 90-degree steps) ===
def rotate_clockwise(mat, k=1):
    """Rotate 90 degrees clockwise k times (k=1 means 90°, k=2 means 180°, etc.)"""
    return np.rot90(mat, -k)

def rotate_counterclockwise(mat, k=1):
    """Rotate 90 degrees counterclockwise k times (k=1 means 90°, k=2 means 180°, etc.)"""
    return np.rot90(mat, k)

# === New: Bit Inversion ===
def invert_bits(mat):
    """Assumes binary matrix of 0s and 1s"""
    return 1 - mat


In [3]:
transformation = shift_diag_up_right

## Task Definition

In [4]:
# Example 1
x1 = np.array([
    [1, 1, 0, 0],
    [1, 1, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
], dtype=np.uint8)

# Example 2
x2 = np.array([
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 1, 1],
    [0, 0, 1, 1]
], dtype=np.uint8)

# Example 3
x3 = np.array([
    [0, 0, 0, 0],
    [0, 0, 1, 1],
    [0, 0, 1, 1],
    [0, 0, 0, 0]
], dtype=np.uint8)

# Example 4
x4 = np.array([
    [1, 0, 0, 0],
    [0, 1, 1, 0],
    [0, 1, 1, 0],
    [0, 0, 0, 1]
], dtype=np.uint8)



# Test
x_test = np.array([
    [0, 1, 1, 0],
    [0, 1, 1, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
], dtype=np.uint8)



train_xs = [x1, x2, x3, x4]
train_ys = [transformation(x) for x in train_xs]

train_xs = np.stack(train_xs)
train_ys = np.stack(train_ys)

y_test = transformation(x_test)


## Abduction

In [5]:
from algorithmic_inference import AlgorithmicAbductionInduction

abducer = AlgorithmicAbductionInduction(num_rules=1_000_000, seed=42, boundary_mode=1)
rule_matches = abducer.abduct_rules(train_xs, train_ys)


/Users/eyujis/Library/Caches/pypoetry/virtualenvs/algorithmic-arc-E897BRAD-py3.11/lib/python3.11/site-packages/pybdm/utils.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream


[Abduction] Simulating rule matches on training pairs...
[Abduction] Found 16 rules that match all training pairs.


## Ranking by Complexity via BDM

In [6]:
# Rank rules by BDM (1D complexity of 512-bit rule encoding)
ranked_rules = abducer.rank_abducted_rules_by_bdm(list(rule_matches.keys()))

# Preview top-ranked rules
for i, (rule, score) in enumerate(ranked_rules[:5]):
    print(f"#{i+1}: Rule {rule} — BDM = {score:.4f}")

[Ranking] Computing 1D BDM for rules...
[Ranking] Done. Ranked 16 rules by simplicity.
#1: Rule 8497382428008698701164205460772613303804465326420334061626673838121904031266149444535151948214417135893881835735856905837991100363459362100758944298624909 — BDM = 1328.2577
#2: Rule 7991261033543717508783091767196498867541296294016571257950915709989123032432100607402255226256251105546136172880892102016323873207599414602192328163925668 — BDM = 1332.1005
#3: Rule 11017158090906613539288879860580644752943514597875536097652089998748471510178749312371967813137652781966162544232650177647376132185594932915578347220616076 — BDM = 1340.2294
#4: Rule 783420599889332279150012935349625184700769182056067616088131133623011853388726675025671606463846359856687962340295920687985874181005273020162468998485445 — BDM = 1349.9523
#5: Rule 12831753464213468352870962050963575240576995519301465818000050330021745965710187972803375662199299779739274828652861391661661563380170744177720846951093878 — BDM = 1350.2527


## Induction

In [7]:
induced_outputs = abducer.induce_outputs_from_rules(x_test, [r for (r, _) in ranked_rules])

In [8]:
k_ctm_scores = abducer.estimate_k_ctm(induced_outputs, total_rules=len(ranked_rules), time_metric='t_min')

print("First 3 estimated conditional complexities:\n")
for i, (y_key, meta) in enumerate(list(k_ctm_scores.items())[:3]):
    matrix = meta['matrix']
    k_val = meta['k_ctm']
    num_rules = meta['num_rules']
    m_prob = meta['m_y_given_x']
    t_min = meta['t_min']
    t_mean = meta['t_mean']

    print(f"Output {i + 1}:")
    print(matrix)
    print(f"K(y'|x) = {k_val:.4f}")
    print(f"# Matching Rules = {num_rules}, m(y|x) = {m_prob:.4e}, t_min = {t_min}, t_mean = {t_mean:.2f}")
    print("-" * 40)



First 3 estimated conditional complexities:

Output 1:
[[0 1 1 0]
 [0 1 1 0]
 [0 0 0 0]
 [0 0 0 0]]
K(y'|x) = inf
# Matching Rules = 16, m(y|x) = 0.0000e+00, t_min = 0, t_mean = 0.00
----------------------------------------
Output 2:
[[0 1 0 0]
 [0 1 1 1]
 [1 0 0 0]
 [1 0 0 1]]
K(y'|x) = 4.0000
# Matching Rules = 1, m(y|x) = 6.2500e-02, t_min = 1, t_mean = 1.00
----------------------------------------
Output 3:
[[1 1 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [1 1 1 0]]
K(y'|x) = 5.0000
# Matching Rules = 1, m(y|x) = 6.2500e-02, t_min = 2, t_mean = 2.00
----------------------------------------


In [9]:
best_hypothesis, sorted_candidates = abducer.select_inductive_hypothesis(k_ctm_scores)

print("Top 10 ranked inductive hypotheses:\n")
for i, (y_key, meta) in enumerate(sorted_candidates[:10]):
    matrix = meta['matrix']
    k_val = meta['k_ctm']
    num_rules = meta['num_rules']
    m_prob = meta['m_y_given_x']
    t_min = meta['t_min']
    t_mean = meta['t_mean']

    print(f"Rank {i + 1}:")
    print(matrix)
    print(f"K(y'|x) = {k_val:.4f}")
    print(f"# Matching Rules = {num_rules}, m(y|x) = {m_prob:.4e}, t_min = {t_min}, t_mean = {t_mean:.2f}")
    print("-" * 40)

Top 10 ranked inductive hypotheses:

Rank 1:
[[0 0 1 1]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 1 1]]
K(y'|x) = 3.0000
# Matching Rules = 16, m(y|x) = 1.0000e+00, t_min = 8, t_mean = 88.19
----------------------------------------
Rank 2:
[[0 1 0 0]
 [0 1 1 1]
 [1 0 0 0]
 [1 0 0 1]]
K(y'|x) = 4.0000
# Matching Rules = 1, m(y|x) = 6.2500e-02, t_min = 1, t_mean = 1.00
----------------------------------------
Rank 3:
[[0 0 0 0]
 [0 0 0 0]
 [1 0 0 1]
 [1 0 0 1]]
K(y'|x) = 4.0000
# Matching Rules = 16, m(y|x) = 1.0000e+00, t_min = 16, t_mean = 109.62
----------------------------------------
Rank 4:
[[1 0 0 0]
 [1 0 0 0]
 [0 0 1 1]
 [1 0 1 1]]
K(y'|x) = 4.0000
# Matching Rules = 1, m(y|x) = 6.2500e-02, t_min = 1, t_mean = 1.00
----------------------------------------
Rank 5:
[[0 0 0 0]
 [1 1 1 0]
 [0 0 1 0]
 [0 1 1 0]]
K(y'|x) = 4.0000
# Matching Rules = 1, m(y|x) = 6.2500e-02, t_min = 1, t_mean = 1.00
----------------------------------------
Rank 6:
[[0 1 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [1 1 0 0]]
K(y'|x) =

In [10]:
for i, (y_key, meta) in enumerate(sorted_candidates):
    if np.array_equal(meta['matrix'], y_test):
        print(f"\nGround-truth y_test found at rank {i + 1} with K(y'|x) = {meta['k_ctm']:.4f}")
        break
else:
    print("\nGround-truth y_test not found among candidates.")



Ground-truth y_test found at rank 1 with K(y'|x) = 3.0000
